In [28]:
# Import necessary libraries

import requests
import lxml.html as lh
import pandas as pd

In [29]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [30]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [31]:
# Parse the first row as our header
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


In [32]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [33]:
# Check the length of each column. Ideally, they should all be the same
[len(C) for (title,C) in col]

[181, 181, 181]

In [34]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [35]:
# Access the top 5 rows of the data frame 
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [36]:
df.columns = ['Borough', 'Neighbourhood','Postcode']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned\n,M1A\n,Not assigned\n
1,Not assigned\n,M2A\n,Not assigned\n
2,Parkwoods\n,M3A\n,North York\n
3,Victoria Village\n,M4A\n,North York\n
4,"Regent Park, Harbourfront\n",M5A\n,Downtown Toronto\n


In [37]:
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,"Regent Park, Harbourfront",M5A,Downtown Toronto


In [38]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

,Postcode,Borough,Neighbourhood
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,"Regent Park, Harbourfront",M5A,Downtown Toronto
5,"Lawrence Manor, Lawrence Heights",M6A,North York
6,"Queen's Park, Ontario Provincial Government",M7A,Downtown Toronto
7,Not assigned,M8A,Not assigned
8,"Islington Avenue, Humber Valley Village",M9A,Etobicoke
9,"Malvern, Rouge",M1B,Scarborough


In [39]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,,,Canadian postal codes
1,Agincourt,M1S,Scarborough
2,"Alderwood, Long Branch",M8W,Etobicoke
3,"Bathurst Manor, Wilson Heights, Downsview North",M3H,North York
4,Bayview Village,M2K,North York
5,"Bedford Park, Lawrence Manor East",M5M,North York
6,Berczy Park,M5E,Downtown Toronto
7,"Birch Cliff, Cliffside West",M1N,Scarborough
8,"Brockton, Parkdale Village, Exhibition Place",M6K,West Toronto
9,"Business reply mail Processing Centre, South C...",M7Y,East Toronto


In [46]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

In [47]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [49]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,,,Canadian postal codes
1,Agincourt,M1S,Scarborough
2,"Alderwood, Long Branch",M8W,Etobicoke
3,"Bathurst Manor, Wilson Heights, Downsview North",M3H,North York
4,Bayview Village,M2K,North York
5,"Bedford Park, Lawrence Manor East",M5M,North York
6,Berczy Park,M5E,Downtown Toronto
7,"Birch Cliff, Cliffside West",M1N,Scarborough
8,"Brockton, Parkdale Village, Exhibition Place",M6K,West Toronto
9,"Business reply mail Processing Centre, South C...",M7Y,East Toronto


In [50]:
# Check the shape of the data frame
df.shape

(181, 3)

In [45]:
df.to_csv(r'df_can.csv')